In [1]:
import cv2


In [2]:
import mediapipe as mp
import numpy as np
from scipy.signal import lfilter
import matplotlib.pyplot as plt

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
hands = mp_hands.Hands(max_num_hands = 2, static_image_mode = False,min_detection_confidence = 0.5)

In [5]:
# storing all the coordinates at different timesteps in a list
list_x = []
list_y = []
list_z = []
list_avg_x = []
list_avg_y = []
list_avg_z = []


In [6]:
# color and thickness of the lines
color = (255,0,0)
thickness = 2
# threshold after which we should start drawing
threshold = 5

# threshold after which we should take base depth
base_threshold = 4



# moving average buffer
k = 5



In [7]:
# canvas on which we are drawing
canvas = np.zeros([512,512,1],dtype=np.uint8)


In [8]:
toggle = 1

     
    

In [9]:
base_depth = 0

In [10]:
lift_threshold = 10
touch_threshold = -25

In [ ]:
cap = cv2.VideoCapture(0)
buffer_count = 0
delay_count = 0
while cap.isOpened():
    success, image = cap.read()
    
        
    if not success:
        print("video capture unsuccessful")
        break
    
    
     
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    
    
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
        
        
        
        buffer_count = buffer_count+1
        hand_landmarks = results.multi_hand_landmarks[0]
        
        # add the coordinates to the list
        list_x.append(int(hand_landmarks.landmark[8].x*512))
        list_y.append(int(hand_landmarks.landmark[8].y*512))
        list_z.append(int(hand_landmarks.landmark[8].z*512))
        
        
        
        
        
        if(buffer_count == k):
            avg_x = sum(list_x[-k:])/k
            avg_y = sum(list_y[-k:])/k
            avg_z = sum(list_z[-k:])/k
            
            list_avg_x.append(int(avg_x))
            list_avg_y.append(int(avg_y))
            list_avg_z.append(int(avg_z))
            buffer_count = 0
            
            #print('{0} {1} '.format(list_avg_z[-1] - base_depth, list_avg_z[-1]))
            print(toggle)
            
            if(len(list_avg_z) == base_threshold):
                base_depth = list_avg_z[-1]
                
            if(list_avg_z[-1] - base_depth > lift_threshold):
                toggle = 0
            if(list_avg_z[-1] - base_depth < touch_threshold):
                toggle = 1
            
            if(len(list_avg_x) > threshold and toggle == 1):
                # get the current coordinates as endpoint
                end_point = (list_avg_x[-1], list_avg_y[-1])
                # get the previous coordinates as the start point
                start_point = (list_avg_x[-2], list_avg_y[-2])
                # draw a line joining these points
                cv2.line(canvas,start_point, end_point,color, thickness)
                
                
                
       
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
    
  
    
    # show the canvas and the webcam image   
    cv2.imshow("Detected Hands", image)
    cv2.imshow("Result", canvas)
    
    
    
    # exit when the user presses 's's
    if cv2.waitKey(5) & 0xFF == ord('s'):
        break
hands.close()
cv2.destroyAllWindows()
cap.release()






1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1


In [ ]:
print(list_z)

In [ ]:
for i in range(len(list_thumb)):
    print(list_thumb[i] - list_z[i])